In [1]:
from call_nlp_api import *
from google.cloud import bigquery
import pandas_gbq

import warnings
warnings.simplefilter("always", category=PendingDeprecationWarning) 
warnings.simplefilter("always", category=DeprecationWarning) 

In [2]:
"""Queries BQ, adds NL API results, and writes back to BQ."""
bq_client = bigquery.Client(location="US", project="ut-goog")
query = """
  SELECT 
    complaint_id, consumer_complaint_narrative
  FROM `bigquery-public-data.cfpb_complaints.complaint_database`
  WHERE consumer_complaint_narrative IS NOT NULL;
"""
dataset = "nl_api"
table = "nl_api_data"
sentiment_path = "./sentiment"

# Load queries into dataframe and load complaint checkpoints.
df = load_query_results_to_df(bq_client, query)

Query results loaded into dataframe!


### Add the sentiment information to BQ table

In [3]:
sentiments = load_checkpoint(sentiment_path)

sentiments = call_nl_api(df, sentiments, sentiment_path,get_sentiment, batch_size=2500)

# Add NL API data to dataframes and write to BQ.
df = add_sentiment_data(df, sentiments)

[./sentiment] count: 2500
Processed 2500 rows in 21.328291416168213 seconds.
[./sentiment] count: 5000
Processed 2500 rows in 21.15940523147583 seconds.
[./sentiment] count: 7500
Processed 2500 rows in 25.100419282913208 seconds.
[./sentiment] count: 10000
Processed 2500 rows in 23.51889419555664 seconds.
[./sentiment] count: 12500
Processed 2500 rows in 21.931458950042725 seconds.
[./sentiment] count: 15000
Processed 2500 rows in 24.94522762298584 seconds.
[./sentiment] count: 17500
Processed 2500 rows in 24.90809726715088 seconds.
[./sentiment] count: 20000
Processed 2500 rows in 22.9310462474823 seconds.
[./sentiment] count: 22500
Processed 2500 rows in 27.83225703239441 seconds.
[./sentiment] count: 25000
Processed 2500 rows in 19.62871217727661 seconds.
[./sentiment] count: 27500
Processed 2500 rows in 18.522119283676147 seconds.
[./sentiment] count: 30000
Processed 2500 rows in 22.98722505569458 seconds.
[./sentiment] count: 32500
Processed 2500 rows in 21.288410663604736 seconds

In [9]:
df.head()

,consumer_complaint_narrative,sentiment_score,sentiment_magnitude
complaint_id,,,
2162217,I received a prepaid {$26.00} XXXX Citibank XX...,-0.5,2.8
2164428,Front of sheet accompanying prepaid card indic...,-0.4,1.9
1509655,I tried to use XXXX prepaid American Express c...,0.0,2.2
1985577,I am a Colorado resident. I have a {$100.00} V...,-0.3,6.2
2415190,I bought a {$400.00} prepaid Mastercard at XXX...,-0.6,4.0


In [7]:
project_id = "ut-goog"
table_id = "nl_api.sentiment_data"

In [12]:
df.to_csv("sentiment_data.csv")

In [13]:
df_sentiments = pd.read_csv("sentiment_data.csv")

In [16]:
df_sentiments.head(50)

,complaint_id,consumer_complaint_narrative,sentiment_score,sentiment_magnitude
0,2162217,I received a prepaid {$26.00} XXXX Citibank XX...,-0.5,2.8
1,2164428,Front of sheet accompanying prepaid card indic...,-0.4,1.9
2,1509655,I tried to use XXXX prepaid American Express c...,0.0,2.2
3,1985577,I am a Colorado resident. I have a {$100.00} V...,-0.3,6.2
4,2415190,I bought a {$400.00} prepaid Mastercard at XXX...,-0.6,4.0
5,2224405,I received a prepaid debit card from Citi bank...,-0.5,1.7
6,1604786,Rush card was under maintance Monday XXXX XXXX...,-0.5,1.0
7,1977697,"Well, I called citi on XXXX/XXXX/16 to determi...",-0.3,6.1
8,1619768,there was suppose to be an update to the rushc...,-0.8,0.8
9,1765923,Per the back of the prepaid visa card I have s...,-0.3,4.2


In [19]:
table_name = "sentiment_data"
df.index = df.index.astype(int)
dataset = bq_client.dataset(dataset)
table_ref = dataset.table(table_name)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: PendingDeprecationWarning: Client.dataset is deprecated and will be removed in a future version. Use a string like 'my_project.my_dataset' or a cloud.google.bigquery.DatasetReference object, instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = "WRITE_TRUNCATE"
job = bq_client.load_table_from_dataframe(df, table_ref, location="US",
                                            job_config=job_config)
job.result()
assert job.state == "DONE"
print("BQ table updated:", table_name)

BQ table updated: sentiment_data
